## Altmap Experiments
### Compare altmap to map eq using networkx

In [3]:
import networkx as nx
import matplotlib.pyplot as plt
from matplotlib.ticker import (AutoMinorLocator)
import numpy as np

# show plots in separate window
%pylab
# load helpers and wrappers
%run helpers.py 

Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


In [43]:
# compute essential cost function values for a barbell network with
# given clique size
def ring_of_cliques_cost(clique_size = 3, number_of_cliques = 5, print_output=False):
    nc = clique_size
    c = number_of_cliques
    
    m = (nc*(nc-1) + 2) * c / 2.0 # number of edges in the network
    p0 = (nc - 1) /(2*m) # stat prob for 'normal' nodes
    pc = nc / (2*m) # stat prob for the connecting nodes

    # initial cost
    J_init = c*(nc-1)*p0*np.log2(1.0-p0) + c*pc*np.log2(1.0-pc)
    
    # maximum independent sets cost
    p1 = c*p0; p2 = (c-1)*p0+pc
    J_ind = (nc-1)*p1*np.log2(1.0-p1) + c*p2*np.log2(1.0-p2)
    
    # ground truth cost
    pi = (nc-1)*p0+pc; pnoti=1.0-pi; pinoti=1.0/(2*m); pii = pi - pinoti
    J_true = -c*(pii*np.log2(pii/pi**2) + pinoti*np.log2(pinoti/(pi*pnoti)))
    
    if print_output:
        print (f"\nRing of {c} cliques with nc = {nc} nodes per clique:\n")
        print (f"Each node a module - cost = {J_init}")
        print (f"Ground truth cost = {J_true}")
        print (f"Independent sets cost = {J_ind}\n")
    
    return J_init, J_true, J_ind

num_cliques = 100
nc_max = 50
nc_list = list(range(3, nc_max + 1))
J_init_list = np.zeros((len(nc_list), 1))
J_true_list = np.zeros_like(J_init_list)
J_ind_list = np.zeros_like(J_init_list)
for i,nc in enumerate(nc_list):
    J_init_list[i], J_true_list[i], J_ind_list[i] = ring_of_cliques_cost(nc, number_of_cliques=num_cliques)


plt.close('all')
fig, ax = plt.subplots()
plt.suptitle(f'Ring of {num_cliques} cliques - cost over clique size')
ax.plot(nc_list, J_true_list, 'x--', label='Ground truth')
ax.plot(nc_list, J_init_list, '^--', label='Each node as a module')
ax.plot(nc_list, J_ind_list, 'o--', label='Maximum independet set')
ax.xaxis.set_minor_locator(AutoMinorLocator())
ax.tick_params(which='both', width=2)
ax.grid(which='both')
ax.set_xlabel('Nodes per clique $n_c$')
ax.set_ylabel('Altmap cost $\mathcal{J}(m)$')
ax.legend()


In [63]:
num_cliques = 20
clique_size = 2
N = num_cliques * clique_size # num nodes
G = nx.ring_of_cliques(num_cliques, clique_size)
G = nx.convert_node_labels_to_integers(G, first_label=1)

# compute ground truth cost
nodes_ids = list(range(1, N+1))
labels = [1] * clique_size
for i in range(2, num_cliques+1):
    labels += [i] * clique_size
communities = dict(zip(nodes_ids, labels))        

cost = altmap_cost(G, communities)
print (f'Ground Truth L = {cost}\n')


# run community detection
# communities_found, num_communities_found = infomap(G, altmap=False)
# communities_found, num_communities_found = infomap(G, altmap=True)
communities_found, num_communities_found = infomap(G, altmap=True, init='sc')

# print results
print (communities_found)
print (f'We found {num_communities_found} communities.')

# cost = altmap_cost(G, communities_found)
# print (f'Achieved cost L = {cost}')


Ground Truth L = -1.1979643381655696

OrderedDict([(1, 5), (2, 5), (3, 5), (4, 5), (5, 5), (6, 5), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 6), (15, 6), (16, 6), (17, 6), (18, 6), (19, 6), (20, 2), (21, 2), (22, 2), (23, 2), (24, 2), (25, 2), (26, 2), (27, 3), (28, 3), (29, 3), (30, 3), (31, 3), (32, 3), (33, 3), (34, 4), (35, 4), (36, 4), (37, 4), (38, 4), (39, 4), (40, 4)])
We found 6 communities.


In [59]:
plt.close('all')
plt.figure()
drawNetwork(G, communities_found)

/home/chri/anaconda3/lib/python3.6/site-packages/networkx/drawing/nx_pylab.py:579: MatplotlibDeprecationWarning: 
The iterable function was deprecated in Matplotlib 3.1 and will be removed in 3.3. Use np.iterable instead.
  if not cb.iterable(width):
